In [2]:
import os
os.getcwd()

'c:\\Users\\gp72h\\Deepcraft\\K_company\\image'

In [26]:
import json
import base64
import anthropic
import os
import json
import pandas as pd

instruction_text="""
## 依頼するタスク ##
１．所定の「項目」が存在する場合に限りそれを抽出してください。
２．それぞれの項目について、抽出結果の信頼性（％）を評価して下さい。
３．抽出した項目は、所定の「出力形式」に従って整形してください。
４．読み取った一番上の行の形式を踏襲して、それ以降下の行も読み取っていってください。

## 抽出する項目 ##

発注番号
-F
-紙質
-巾
-流れ
-丁取り
-罫線1
-罫線2
-罫線3
-罫線4
-罫線5
-罫線6
-罫線7
-罫線8
-罫線9
-罫線10
-罫線11
-罫線12
-罫線13
-数量
-納期
-備考


## 出力形式 ##
-項目の抽出結果はcsv形式で出力してください。
-表が読み取れずに形成できない場合は"不明"で埋めて下さい。


## 指示とアドバイス ##
- ステップバイステップで考えてください。
- 深呼吸して考えてください。
- この分析は私のキャリアにとってとても重要です。
- 自分の能力を信じて、もっと上を目指してください。
- このタスクにおいて、最高の結果を出すために、追加の情報が必要な場合は、逆質問をしてください。
- 不明確な項目は"不明"と出力すること
- 数値は整数で出力すること。
- 整数に小数点がついてる場合は切り捨てること。
- 罫線の列は例えば、"158/145/158"もしくは"(158+145+158=461)"のようになっていることが多いです。これを認識してください。
- 罫線は例えば"158/145/158"と読み取ったとき罫線1:"158",罫線2:"145",罫線3:"158"のように分けて出力してください。
- 罫線で例えば"158/145/158=461"と読み取った場合、罫線1:"158",罫線2:"145",罫線3:"158",罫線4:"不明",....罫線13:"不明"のように出力してください。
- 罫線は例えば"(158+145+158=461)"と読み取ったとき罫線1:"158",罫線2:"145",罫線3:"158"のように分けて出力してください。
- 罫線は例えば"158+145+158=461"と読み取ったとき罫線1:"158",罫線2:"145",罫線3:"158"のように分けて出力してください。=461の部分は無視してください。
- 罫線は経線13まで出力してほしいが後半は普通からのデータとなっている。ここは"不明"として出力して下さい。
- 罫線という項目において"="という文字を認識したらそれ以降の数字は出力しないでください。
- 丁取りは１桁の整数であることに注意して読み取ってください。
- 出力するjsonデータが複数ある場合は信憑性の最も高いものを1つだけ出力して下さい。
- "スリッタ寸法"というカラムがある場合、それは"罫線"と解釈してください。
- "F"と"フルート"は同じと解釈してください。
- "フルート"が項目としてある場合、"F"と解釈して出力してください。
- "F"または"フルート"または"段"のカラムは"A"か"B"か"W"の３つのいずれかです。したがってそれ以外のものを認識しようとしている場合それは間違いです。認識したものを１つ出力してください。
- "KSSK5"や"KSK5"や"C5SC5"などは"紙質"を意味します。
- "納期"は例えば"1/19"などの形式で書かれていることが多いです。
- "1250x1240"は基本"巾x流れ"を表しています。この場合は巾:1250、流れ:1240のようにわけてください。
- 抽出する項目は順不同である。最も妥当な項目に内容が対応するように読み取ってください。
- 入力の画像に例えば"No."の項目がない場合これは出力しないものとする。
- このタスクにおいて、最高の結果を出すために、追加の情報が必要な場合は、逆質問をしてください。
- 同じ列には同じタイプの形式のデータが入るようにしてください。
"""

client = anthropic.Anthropic(
    api_key=os.environ.get("CLAUDE3_API"),
)

# 画像をbase64にエンコードする
def get_base64_encoded_image(image_path: str) -> str:
    with open(image_path, "rb") as image_file:
        binary_data = image_file.read()
        base_64_encoded_data = base64.b64encode(binary_data)
        base64_string = base_64_encoded_data.decode('utf-8')
        return base64_string
    

with open('20240118081021110_0.json', 'r', encoding='utf-8') as file:  # テキストモードでファイルを開く
    response = json.load(file)

result_str = ""
for line in response["lines"]:
    result_str += line["text"]

prompt = result_str
    
image_path = "20240118081021110.jpg"
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=2048,
    system = instruction_text,
    messages = [
    {  
        "role": 'user',
        "content": [
            {"type": "image", "source": {"type": "base64", "media_type": "image/jpeg", "data": get_base64_encoded_image(image_path)}},
            {"type": "text", "text": f"{prompt}"}],
    }
        ]
)
print(message.content[0].text)



発注番号,フルート,紙質,巾,流れ,丁取り,罫線,数量,納期,備考
250242,W,K5SK5,1250,1250,,不明,17,1/19,
250243,A,K5K5,1250,1240,,不明,17,1/19,
250244,A,K5K5,1200,1550,不明,"(124+340+124=588)",101,1/19,
250245,A,K5K5,1300,1772,,不明,12,1/19,
250246,A,K5K5,1350,995,,不明,21,1/19,
250248,W,C5SC5,1250,1507,,不明,16,1/19,

信頼性評価:
発注番号: 100%
フルート: 100% 
紙質: 90%
巾: 100%
流れ: 100%
丁取り: 0%
罫線: 20% 
数量: 100%
納期: 100%
備考: 0%

本データは表形式の情報を高い精度で読み取れています。一部の項目は情報が不明ですが、発注番号、フルート、巾、流れ、数量、納期については100%の信頼性で抽出できています。罫線については250244行にのみ情報がありますが、その他の行では不明です。丁取りと備考に関しては情報がないため信頼性0%となっています。


In [2]:
import json
with open('20240118081021110_0.json', 'r', encoding='utf-8') as file:  # テキストモードでファイルを開く
    response = json.load(file)

result_str = ""
for line in response["lines"]:
    result_str += line["text"]

result_str

'2024年 1月18日 8時12分二番付材NO. 7278P.2600 久門紙器工業 様シート 発注 書三協荷材株式会社2024年 1月 18日 頁 1/ 1TEL (072) 883-2525FAX (072) 883-2529担当者:三浦孝之発注No. フルート材 質巾 ×流れスリッタ寸法枚数納期備 考250242K5SK51250 × 1250171/19250243AK5K51250 × 1240171/19250244AK5K51200 × 1550(124+340+124) = 5881011/19250245AK5K51300 × 177212 1/19250246 AK5K51350 × 99521 1/19250248 WC5SC51250 × 150716 1/19.'